# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

In [5]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
serper_api_key = os.getenv("SERPER_API_KEY")

In [6]:
# Creating Agents
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile", # ICP - used in sales!
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [10]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [11]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [12]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [15]:
class SentimentAnalysisTool(BaseTool): # every tool needs name and description
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        """Create _run function inside the class, and any custom code that
        you want can go here."""
        return "positive"

In [16]:
sentiment_analysis_tool = SentimentAnalysisTool() # now we can use this to pass to our agents and tasks

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [17]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [18]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [19]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [2024-07-29 14:52:18][DEBUG]: == Working Agent: Sales Representative
 [2024-07-29 14:52:18][INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To create a comprehensive report on DeepLearningAI, I need to gather information about the company, key personnel, recent milestones, and identified needs. I will use the available tools strategically to compile this data.

First, I will search the internet to gather general information about DeepLearningAI, including company background, recent news, and key perso

In [22]:
from IPython.display import Markdown
Markdown(result.raw)

Here are three personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng:

### Email Draft 1

Subject: Empowering “Generative AI for Everyone” with CrewAI’s Cutting-Edge Solutions

Dear Dr. Ng,

I hope this message finds you well. Congratulations on the successful launch of “Generative AI for Everyone” on Coursera! This initiative is a testament to DeepLearning.AI’s unwavering commitment to democratizing AI education and empowering individuals worldwide.

At CrewAI, we share your passion for advancing AI and making it accessible to all. Our suite of AI-driven solutions is designed to complement and enhance educational platforms, ensuring that learners not only acquire knowledge but also engage with the material in meaningful ways.

**How CrewAI Can Support “Generative AI for Everyone”**:
1. **Interactive Learning Experiences**: Our AI-powered tools can create interactive and personalized learning experiences that adapt to each student's unique needs and pace, ensuring a deeper understanding of complex AI concepts.
2. **Real-Time Feedback and Assessment**: With CrewAI, you can provide real-time feedback and assessments, helping students stay on track and achieve their learning goals more effectively.
3. **Scalable Content Delivery**: Our scalable solutions ensure that your course content can reach a global audience without compromising on quality or engagement.

We would love the opportunity to discuss how CrewAI can support DeepLearning.AI’s mission and enhance your existing offerings. Are you available for a brief call next week?

Thank you for your time and consideration. I look forward to the possibility of collaborating with you.

Best regards,  
[Your Name]  
Lead Sales Representative  
CrewAI

---

### Email Draft 2

Subject: Enhancing DeepLearning.AI’s Educational Impact with CrewAI’s Innovative Tools

Dear Dr. Ng,

I hope you are doing well. I am thrilled to see the launch of “Generative AI for Everyone” on Coursera. This course is a significant milestone for DeepLearning.AI, further solidifying your role as a leader in AI education.

At CrewAI, we are equally committed to advancing AI and education. We believe that our innovative solutions can greatly enhance the learning experience offered by your new course. By integrating CrewAI’s tools, we can ensure that your learners benefit from the most engaging and effective educational technology available.

**Key Benefits of CrewAI’s Solutions**:
1. **Adaptive Learning**: Our AI-driven platform personalizes content delivery, ensuring each learner receives guidance tailored to their unique progress and understanding.
2. **Enhanced Engagement**: Interactive modules and real-time feedback keep learners motivated and actively participating throughout the course.
3. **Efficient Scaling**: Our robust infrastructure supports large-scale deployment, making it easier to reach and teach a global audience.

I would love to discuss how CrewAI can support the continued success of “Generative AI for Everyone” and explore potential collaboration opportunities. Could we schedule a call next week to discuss this further?

Thank you for your attention. I look forward to the opportunity to work together.

Best regards,  
[Your Name]  
Lead Sales Representative  
CrewAI

---

### Email Draft 3

Subject: Unleashing the Full Potential of “Generative AI for Everyone” with CrewAI

Dear Dr. Ng,

I hope this email finds you in great spirits. Congratulations on the recent launch of “Generative AI for Everyone” on Coursera! This initiative is a remarkable achievement and a clear reflection of DeepLearning.AI’s dedication to making AI education accessible to all.

At CrewAI, we are passionate about leveraging AI to create transformative educational experiences. Our suite of solutions is designed to enhance the learning journey, making it more interactive, personalized, and impactful.

**How CrewAI Can Add Value to Your Course**:
1. **Personalized Learning Pathways**: Our AI algorithms can customize learning paths based on individual student performance, ensuring a more personalized and effective learning experience.
2. **Interactive Engagement**: By incorporating interactive elements and real-time feedback, CrewAI’s tools can significantly boost student engagement and comprehension.
3. **Global Reach with High Quality**: Our scalable solutions ensure that your high-quality content is delivered seamlessly to a global audience, maintaining engagement and educational standards.

I am eager to discuss how CrewAI can support the incredible work being done at DeepLearning.AI and explore ways we can collaborate to enhance your course offerings. Would you be available for a brief call next week?

Thank you for considering this opportunity. I look forward to the possibility of working together.

Warm regards,  
[Your Name]  
Lead Sales Representative  
CrewAI

## Exception Handling:
### Option 1: 
* Framework stops execution (you can choose how to handle the exceptions, such as restart execution)
### Option 2:
* Framework continues execution (CrewAI is designed this way)